<h1>Popularity Based recommendation system</h1>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [2]:
ratings = pd.read_csv('ratings_Electronics (1).csv')
ratings_df = ratings.copy()

In [3]:
# Let's rename the columns 
ratings_df.columns = ['userId', 'productId', 'Rating', 'timestamp']
ratings_df

,userId,productId,Rating,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600
...,...,...,...,...
7824476,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824477,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824478,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824479,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [4]:
total_votes = pd.DataFrame(ratings_df.groupby('productId')['Rating'].count().reset_index())
total_votes.columns = ['productId', 'vote_count']
total_votes = total_votes.merge(ratings_df, on = "productId")

mean_rating = total_votes.groupby('productId')['Rating'].mean().reset_index()
mean_rating = mean_rating.merge(total_votes, on = "productId")
mean_rating.columns = ['productId', 'avg rating', 'vote_count', 'userId', 'rating', 'timestamp']

In [5]:
mean_rating.shape

(7824481, 6)

In [6]:
v = mean_rating['vote_count']
m = mean_rating['vote_count'].quantile(0.95)
R = mean_rating['avg rating']
C = mean_rating['rating'].mean()

mean_rating = mean_rating[mean_rating['vote_count'] >= m]

In [7]:
mean_rating.shape

(392689, 6)

In [8]:
recommended_products = mean_rating.drop(['userId', 'timestamp', 'rating'], axis = 1)
recommended_products.drop_duplicates(inplace = True)

In [9]:
recommended_products.shape

(63, 3)

In [10]:
def recommendation(x):
    v = x['vote_count']
    R = x['avg rating']
    return (v/(v+m) * R) + (m/(m+v) * C)
recommended_products['score'] = recommended_products.apply(recommendation, axis = 1)

In [11]:
recommended_products.shape

(63, 4)

In [12]:
recommended_products.sort_values(by ="score", ascending = False, inplace = True)

In [13]:
recommended_products.shape

(63, 4)

In [14]:
recommended_products.head(10)

,productId,avg rating,vote_count,score
1814399,B0019EHU8G,4.754497,12285,4.586158
1133243,B000LRMS66,4.756627,8715,4.538880
3313475,B003ES5ZUU,4.704749,10276,4.524961
5869187,B007R5YDYA,4.690926,9907,4.509915
2991140,B00316263Y,4.735609,5038,4.433981
5592209,B0074BW614,4.491504,18244,4.412462
2883523,B002V88HFE,4.666313,4708,4.382755
1312290,B000QUUFRW,4.688878,4037,4.369776
3426065,B003LR7ME6,4.508484,8840,4.364791
5941359,B007WTAJTO,4.424005,14172,4.340541


In [15]:
# Amazon api
Asin = pd.DataFrame(recommended_products.head(15))
Asin=Asin.reset_index().drop('index',axis=1)
Asin

,productId,avg rating,vote_count,score
0,B0019EHU8G,4.754497,12285,4.586158
1,B000LRMS66,4.756627,8715,4.538880
2,B003ES5ZUU,4.704749,10276,4.524961
3,B007R5YDYA,4.690926,9907,4.509915
4,B00316263Y,4.735609,5038,4.433981
5,B0074BW614,4.491504,18244,4.412462
6,B002V88HFE,4.666313,4708,4.382755
7,B000QUUFRW,4.688878,4037,4.369776
8,B003LR7ME6,4.508484,8840,4.364791
9,B007WTAJTO,4.424005,14172,4.340541


In [16]:
Asin['API']=Asin.productId.apply(lambda x:'https://amazon-scraper.tprojects.workers.dev/product/dp/'+x )
#Asin.productId.to_list
Asin

,productId,avg rating,vote_count,score,API
0,B0019EHU8G,4.754497,12285,4.586158,https://amazon-scraper.tprojects.workers.dev/p...
1,B000LRMS66,4.756627,8715,4.538880,https://amazon-scraper.tprojects.workers.dev/p...
2,B003ES5ZUU,4.704749,10276,4.524961,https://amazon-scraper.tprojects.workers.dev/p...
3,B007R5YDYA,4.690926,9907,4.509915,https://amazon-scraper.tprojects.workers.dev/p...
4,B00316263Y,4.735609,5038,4.433981,https://amazon-scraper.tprojects.workers.dev/p...
5,B0074BW614,4.491504,18244,4.412462,https://amazon-scraper.tprojects.workers.dev/p...
6,B002V88HFE,4.666313,4708,4.382755,https://amazon-scraper.tprojects.workers.dev/p...
7,B000QUUFRW,4.688878,4037,4.369776,https://amazon-scraper.tprojects.workers.dev/p...
8,B003LR7ME6,4.508484,8840,4.364791,https://amazon-scraper.tprojects.workers.dev/p...
9,B007WTAJTO,4.424005,14172,4.340541,https://amazon-scraper.tprojects.workers.dev/p...


In [17]:
headers = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
    }
name=[]
image=[]
prod_link=[]
productId=[]
count=0
for link in Asin.API:
  payload = requests.get(link,headers=headers).json()
  if payload['product_detail']== None:
    continue
  productId.append(link[-10:]) # ASIN Number are 10-digits
  name.append(payload['product_detail']['name'])
  image.append(payload['product_detail']['image'])
  prod_link.append(payload['product_detail']['product_link'])
  count=count+1
  print(count,name[count-1])
  if count==10:
    break
    

1 Mediabridge ULTRA Series HDMI Cable (6 Feet) - High-Speed Supports Ethernet, 3D and Audio Return [Newest Standard]
2 Garmin Portable Friction Mount
3 BlueRigger High Speed HDMI Cable with Ethernet - Supports 3D, 4K 60Hz, Audio Return - Latest Version (6.6 Feet / 2 Meters)
4 eneloop SEC-CSPACER4PK C Size Spacers for use with AA Battery Cells
5 SanDisk Extreme 4GB UHS-I SDHC Memory Card (SDSDX-004G-X46)
6 Snugg Leather Case with Flip Stand for Apple iPad (Black)
7 Sandisk Mobile Ultra 64GB Class 10 micro SDXC Memory Card with SD Adaptor Upto 30 Mbps speed (SDSDQUA-064G-U46A)
8 Bear Motion (Trademark) Genuine Leather Case for Apple Ipad 1 (First Generation) - Black
9 DVI Gear HDMI Cable 2M 6 feet
10 Securifi Almond - (3 Minute Setup) Touchscreen Wireless Router/Range Extender


In [18]:
print(name,image,prod_link,productId)
zipped=list(zip(productId,name,prod_link,image))
temp=pd.DataFrame(zipped,columns=['productId','name','prod_link','image'])

['Mediabridge ULTRA Series HDMI Cable (6 Feet) - High-Speed Supports Ethernet, 3D and Audio Return [Newest Standard]', 'Garmin Portable Friction Mount', 'BlueRigger High Speed HDMI Cable with Ethernet - Supports 3D, 4K 60Hz, Audio Return - Latest Version (6.6 Feet / 2 Meters)', 'eneloop SEC-CSPACER4PK C Size Spacers for use with AA Battery Cells', 'SanDisk Extreme 4GB UHS-I SDHC Memory Card (SDSDX-004G-X46)', 'Snugg Leather Case with Flip Stand for Apple iPad (Black)', 'Sandisk Mobile Ultra 64GB Class 10 micro SDXC Memory Card with SD Adaptor Upto 30 Mbps speed (SDSDQUA-064G-U46A)', 'Bear Motion (Trademark) Genuine Leather Case for Apple Ipad 1 (First Generation) - Black', 'DVI Gear HDMI Cable 2M 6 feet', 'Securifi Almond - (3 Minute Setup) Touchscreen Wireless Router/Range Extender'] ['https://m.media-amazon.com/images/I/A1HVSAiVnAL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/81kmeywBvIL._SL1500_.jpg', 'https://m.media-amazon.com/images/I/81pyQdiNt7S._SL1500_.jpg', 'https://m.

In [19]:
temp

,productId,name,prod_link,image
0,B0019EHU8G,Mediabridge ULTRA Series HDMI Cable (6 Feet) -...,https://www.amazon.in/dp/B0019EHU8G,https://m.media-amazon.com/images/I/A1HVSAiVnA...
1,B000LRMS66,Garmin Portable Friction Mount,https://www.amazon.in/dp/B000LRMS66,https://m.media-amazon.com/images/I/81kmeywBvI...
2,B00316263Y,BlueRigger High Speed HDMI Cable with Ethernet...,https://www.amazon.in/dp/B00316263Y,https://m.media-amazon.com/images/I/81pyQdiNt7...
3,B002V88HFE,eneloop SEC-CSPACER4PK C Size Spacers for use ...,https://www.amazon.in/dp/B002V88HFE,https://m.media-amazon.com/images/I/61htqNyxdM...
4,B000QUUFRW,SanDisk Extreme 4GB UHS-I SDHC Memory Card (SD...,https://www.amazon.in/dp/B000QUUFRW,https://m.media-amazon.com/images/I/81gXK5ezxa...
5,B003LR7ME6,Snugg Leather Case with Flip Stand for Apple i...,https://www.amazon.in/dp/B003LR7ME6,https://m.media-amazon.com/images/I/71Nqc3NpU8...
6,B007WTAJTO,Sandisk Mobile Ultra 64GB Class 10 micro SDXC ...,https://www.amazon.in/dp/B007WTAJTO,https://m.media-amazon.com/images/I/61h7zFbXLN...
7,B003ZUIHY8,Bear Motion (Trademark) Genuine Leather Case f...,https://www.amazon.in/dp/B003ZUIHY8,https://m.media-amazon.com/images/I/A1LUbviheK...
8,B0002L5R78,DVI Gear HDMI Cable 2M 6 feet,https://www.amazon.in/dp/B0002L5R78,https://m.media-amazon.com/images/I/81eK2Esijt...
9,B0087NZ31S,Securifi Almond - (3 Minute Setup) Touchscreen...,https://www.amazon.in/dp/B0087NZ31S,https://m.media-amazon.com/images/I/71p-9GjdBW...


In [20]:
df_final=temp.merge(Asin,on='productId',how='inner')
df_final

,productId,name,prod_link,image,avg rating,vote_count,score,API
0,B0019EHU8G,Mediabridge ULTRA Series HDMI Cable (6 Feet) -...,https://www.amazon.in/dp/B0019EHU8G,https://m.media-amazon.com/images/I/A1HVSAiVnA...,4.754497,12285,4.586158,https://amazon-scraper.tprojects.workers.dev/p...
1,B000LRMS66,Garmin Portable Friction Mount,https://www.amazon.in/dp/B000LRMS66,https://m.media-amazon.com/images/I/81kmeywBvI...,4.756627,8715,4.538880,https://amazon-scraper.tprojects.workers.dev/p...
2,B00316263Y,BlueRigger High Speed HDMI Cable with Ethernet...,https://www.amazon.in/dp/B00316263Y,https://m.media-amazon.com/images/I/81pyQdiNt7...,4.735609,5038,4.433981,https://amazon-scraper.tprojects.workers.dev/p...
3,B002V88HFE,eneloop SEC-CSPACER4PK C Size Spacers for use ...,https://www.amazon.in/dp/B002V88HFE,https://m.media-amazon.com/images/I/61htqNyxdM...,4.666313,4708,4.382755,https://amazon-scraper.tprojects.workers.dev/p...
4,B000QUUFRW,SanDisk Extreme 4GB UHS-I SDHC Memory Card (SD...,https://www.amazon.in/dp/B000QUUFRW,https://m.media-amazon.com/images/I/81gXK5ezxa...,4.688878,4037,4.369776,https://amazon-scraper.tprojects.workers.dev/p...
5,B003LR7ME6,Snugg Leather Case with Flip Stand for Apple i...,https://www.amazon.in/dp/B003LR7ME6,https://m.media-amazon.com/images/I/71Nqc3NpU8...,4.508484,8840,4.364791,https://amazon-scraper.tprojects.workers.dev/p...
6,B007WTAJTO,Sandisk Mobile Ultra 64GB Class 10 micro SDXC ...,https://www.amazon.in/dp/B007WTAJTO,https://m.media-amazon.com/images/I/61h7zFbXLN...,4.424005,14172,4.340541,https://amazon-scraper.tprojects.workers.dev/p...
7,B003ZUIHY8,Bear Motion (Trademark) Genuine Leather Case f...,https://www.amazon.in/dp/B003ZUIHY8,https://m.media-amazon.com/images/I/A1LUbviheK...,4.636664,3933,4.338127,https://amazon-scraper.tprojects.workers.dev/p...
8,B0002L5R78,DVI Gear HDMI Cable 2M 6 feet,https://www.amazon.in/dp/B0002L5R78,https://m.media-amazon.com/images/I/81eK2Esijt...,4.448614,9487,4.328505,https://amazon-scraper.tprojects.workers.dev/p...
9,B0087NZ31S,Securifi Almond - (3 Minute Setup) Touchscreen...,https://www.amazon.in/dp/B0087NZ31S,https://m.media-amazon.com/images/I/71p-9GjdBW...,4.581727,4203,4.318876,https://amazon-scraper.tprojects.workers.dev/p...
